In [217]:
#Arabidopsis  RNA-seq and protein level data for 30 different tissues was taken from Mergner et al 2020 https://www.nature.com/articles/s41586-020-2094-2
#the 2nd sheet (gene expression) of supplementary table 2 was saved as a csv. Location: ../../data/genes/mergner2020_RNA-seq_CVs.csv

#this notebook calculates the mean and SD and then CV (coefficient of variation, SD/mean) allowing ranking of genes by how stably expressed they are

##can us https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.variation.html for calculating cv in future

In [1]:
import pandas as pd

In [2]:
def calculate_cv(df, data_type):
    """function to calculate the cv (SD/mean) of the RNA or protein_df. Provide the dataframe containing columns of interest."""

    #reverse log transform
    df2 = df.copy()
    df2.iloc[:,3:] = df.iloc[:,3:].apply(lambda x: 2**x)
    #change NaN values to 0 expression (mRNA quantities are displayed as TPM and a cut-off value of 1 TPM was used as lower limit of detection across all samples)
    df = df2.copy()
    df.iloc[:,3:] = df2.iloc[:,3:].fillna(0)

    #calculate mean
    df = df.assign(mean=df.iloc[:,3:].mean(axis=1))
    df.rename(columns={'mean':f'{data_type}_mean'}, inplace=True)
    #calculate SD
    df = df.assign(sd=df.iloc[:,3:].std(axis=1))
    df.rename(columns={'sd':f'{data_type}_sd'}, inplace=True)
    #calculate CV
    df = df.assign(cv=df[f'{data_type}_sd'] / df[f'{data_type}_mean'])
    df.rename(columns={'cv':f'{data_type}_cv'}, inplace=True)
    
    return df
    

In [3]:
def save_csv(input_df, output):
    """function to extract useful columns and save them as a csv"""
    df = input_df.iloc[:, [0,1,2,35]]
    #remove nan
    df2 = df[df.iloc[:,3].notnull()]
    df_sorted = df2.sort_values(df.columns[3]).reset_index(drop=True)
    df_sorted.to_csv(output, index=False)
    
    

In [5]:
original_csv = '../../data/genes/mergner2020_RNA-seq_CVs.csv'
df = pd.read_csv(original_csv, header=1)
RNA_df = df[['AGI code', 'transcription (iTAK)_class', 'transcription (iTAK)_family', 'TPM_SP', 'TPM_PT','TPM_ST','TPM_CP','TPM_SQ','TPM_EB','TPM_SD','TPM_SDIMB','TPM_P','TPM_ND','TPM_IND','TPM_CLLF','TPM_LFD','TPM_LFP','TPM_LFPT','TPM_SCLF','TPM_TPM_RT','TPM_FLPD','TPM_FL','TPM_SQSP',
             'TPM_SQV','TPM_RKD2','TPM_CIM','TPM_CC3','TPM_CC10','TPM_CT','TPM_HY','TPM_RTTP','TPM_RTUZ','TPM_CTSAM'
]]
protein_df = df[['AGI code', 'transcription (iTAK)_class', 'transcription (iTAK)_family','iBAQ_SP','iBAQ_PT','iBAQ_ST','iBAQ_CP','iBAQ_SQ','iBAQ_EB','iBAQ_SD','iBAQ_SDIMB',
                 'iBAQ_P','iBAQ_ND','iBAQ_IND','iBAQ_CLLF','iBAQ_LFD','iBAQ_LFP','iBAQ_LFPT','iBAQ_SCLF','iBAQ_iBAQ_RT','iBAQ_FLPD','iBAQ_FL','iBAQ_SQSP','iBAQ_SQV','iBAQ_RKD2','iBAQ_CIM',
                 'iBAQ_CC3','iBAQ_CC10','iBAQ_CT','iBAQ_HY','iBAQ_RTTP','iBAQ_RTUZ','iBAQ_CTSAM'
]]

In [6]:
RNA_output = '../../data/genes/RNA_CVs.csv'
protein_output = '../../data/genes/proteincontent_CVs.csv'

In [8]:
rna_reversetransformed = calculate_cv(RNA_df, 'RNA')
protein_reversetransformed = calculate_cv(protein_df, 'proteincontent')

In [12]:
rna_reversetransformed.sort_values('RNA_cv', inplace=True, ignore_index=True)
#filter NaN
#remove NaNs in RNA_cv column
filtered_df_RNA = rna_reversetransformed[rna_reversetransformed.RNA_cv.notnull()].copy()

In [13]:
filtered_df_RNA

,AGI code,transcription (iTAK)_class,transcription (iTAK)_family,TPM_SP,TPM_PT,TPM_ST,TPM_CP,TPM_SQ,TPM_EB,TPM_SD,...,TPM_CC3,TPM_CC10,TPM_CT,TPM_HY,TPM_RTTP,TPM_RTUZ,TPM_CTSAM,RNA_mean,RNA_sd,RNA_cv
0,AT1G79940,NaN,NaN,44.428699,34.976645,28.971450,30.249303,34.203214,19.686572,32.748797,...,38.312148,27.573299,35.094426,49.643055,42.978404,46.186202,40.674988,37.846483,6.854707,0.181119
1,AT2G32170,NaN,NaN,17.817339,23.258401,18.714570,21.584144,21.305441,10.661807,28.337096,...,26.683204,18.964902,17.357172,19.680978,23.701558,25.505739,21.913834,21.017811,4.054579,0.192912
2,AT3G50860,NaN,NaN,10.678225,10.231317,10.311126,12.429962,16.588712,9.672709,12.846380,...,10.703643,10.892249,9.814740,12.620437,16.191784,14.219709,10.539978,11.816719,2.478108,0.209712
3,AT3G49850,TF,MYB-related,13.058796,10.494198,8.722973,16.511852,21.725538,19.318618,11.849711,...,12.240665,10.313842,15.668118,14.970391,14.313355,15.452303,13.546610,13.709074,2.901376,0.211639
4,AT5G25760,NaN,NaN,61.892814,76.523980,67.424920,51.578932,79.743222,78.147618,50.963767,...,58.833257,50.112201,48.939586,57.528303,73.263870,61.959346,65.353835,63.955337,13.632518,0.213157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25152,AT5G45460,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.279921,1.507418,5.385165
25153,AT1G11520,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.595148,3.204968,5.385165
25154,AT5G48890,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041058,0.221106,5.385165
25155,AT3G59170,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.392904,2.115854,5.385165


In [11]:
rna_reversetransformed

,AGI code,transcription (iTAK)_class,transcription (iTAK)_family,TPM_SP,TPM_PT,TPM_ST,TPM_CP,TPM_SQ,TPM_EB,TPM_SD,...,TPM_CC3,TPM_CC10,TPM_CT,TPM_HY,TPM_RTTP,TPM_RTUZ,TPM_CTSAM,RNA_mean,RNA_sd,RNA_cv
0,AT1G79940,NaN,NaN,44.428699,34.976645,28.971450,30.249303,34.203214,19.686572,32.748797,...,38.312148,27.573299,35.094426,49.643055,42.978404,46.186202,40.674988,37.846483,6.854707,0.181119
1,AT2G32170,NaN,NaN,17.817339,23.258401,18.714570,21.584144,21.305441,10.661807,28.337096,...,26.683204,18.964902,17.357172,19.680978,23.701558,25.505739,21.913834,21.017811,4.054579,0.192912
2,AT3G50860,NaN,NaN,10.678225,10.231317,10.311126,12.429962,16.588712,9.672709,12.846380,...,10.703643,10.892249,9.814740,12.620437,16.191784,14.219709,10.539978,11.816719,2.478108,0.209712
3,AT3G49850,TF,MYB-related,13.058796,10.494198,8.722973,16.511852,21.725538,19.318618,11.849711,...,12.240665,10.313842,15.668118,14.970391,14.313355,15.452303,13.546610,13.709074,2.901376,0.211639
4,AT5G25760,NaN,NaN,61.892814,76.523980,67.424920,51.578932,79.743222,78.147618,50.963767,...,58.833257,50.112201,48.939586,57.528303,73.263870,61.959346,65.353835,63.955337,13.632518,0.213157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25280,AT5G59190,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25281,AT5G60080,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25282,AT5G60090,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25283,AT5G60510,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [14]:
save_csv(filtered_df_RNA, RNA_output)
#save_csv(protein_reversetransformed, protein_output)

In [8]:
rna_reversetransformed

NameError: name 'rna_reversetransformed' is not defined

In [235]:
protein_reersetransformed

,AGI code,transcription (iTAK)_class,transcription (iTAK)_family,iBAQ_SP,iBAQ_PT,iBAQ_ST,iBAQ_CP,iBAQ_SQ,iBAQ_EB,iBAQ_SD,...,iBAQ_CC3,iBAQ_CC10,iBAQ_CT,iBAQ_HY,iBAQ_RTTP,iBAQ_RTUZ,iBAQ_CTSAM,proteincontent_mean,proteincontent_sd,proteincontent_cv
0,AT1G01010,TF,NAC,2.215542e+05,0.000000e+00,3.270319e+06,1.578287e+05,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,6.755954e+05,0.000000e+00,0.000000e+00,1.693880e+06,9.396083e+05,0.000000e+00,4.466096e+05,7.622711e+05,1.706795
1,AT1G01020,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
2,AT1G01030,TF,B3,1.205416e+06,2.907605e+06,1.677787e+05,1.027709e+06,7.518055e+05,2.134591e+05,0.000000e+00,...,0.000000e+00,0.000000e+00,4.658158e+05,0.000000e+00,3.112253e+05,0.000000e+00,1.117389e+06,3.349197e+05,6.204351e+05,1.852489
3,AT1G01040,NaN,NaN,1.738881e+05,1.190305e+06,6.004148e+05,6.577592e+05,5.272033e+05,4.609018e+05,2.409376e+05,...,3.509174e+06,7.077597e+05,3.730217e+05,2.896598e+05,1.144432e+06,6.138387e+05,7.597683e+05,6.187560e+05,6.275489e+05,1.014211
4,AT1G01050,NaN,NaN,3.084275e+06,2.252351e+06,2.879526e+06,7.547294e+05,1.386100e+06,2.009214e+06,0.000000e+00,...,0.000000e+00,5.216678e+06,0.000000e+00,4.430076e+06,3.252235e+06,0.000000e+00,5.085295e+06,2.796509e+06,2.448531e+06,0.875567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25280,ATMG01350,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25281,ATMG01360,NaN,NaN,4.464515e+07,5.333946e+07,1.087319e+08,3.601514e+07,4.733487e+07,2.196495e+07,1.357642e+07,...,6.560034e+07,3.240246e+07,1.693847e+07,3.733176e+07,3.720002e+07,6.919670e+07,1.538691e+07,4.064264e+07,3.757992e+07,0.924643
25282,ATMG01370,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25283,ATMG01400,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
